# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=poklpokl")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]
print(filepath)

/home/intelligence/hustle/song_play_analysis/data/song_data/A/A/C/TRAACZK128F4243829.json


In [6]:
print(filepath)
df = pd.read_json(filepath, lines=True)
df.head()

/home/intelligence/hustle/song_play_analysis/data/song_data/A/A/C/TRAACZK128F4243829.json


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARGUVEV1187B98BA17,NaN,NaN,,Sierra Maestra,SOGOSOV12AF72A285E,¿Dónde va Chichi?,313.12934,1997


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = []
song_data = [df.at[0, "song_id"], df.at[0, "title"], df.at[0, "artist_id"], int(df.at[0, "year"]), df.at[0, "duration"]]
print(song_data)

['SOGOSOV12AF72A285E', '¿Dónde va Chichi?', 'ARGUVEV1187B98BA17', 1997, 313.12934]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOGOSOV12AF72A285E) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = [df.at[0, "artist_id"], df.at[0, "artist_name"], df.at[0, "artist_location"], df.at[0, "artist_latitude"], df.at[0, "artist_latitude"]]
print(artist_data)

['ARGUVEV1187B98BA17', 'Sierra Maestra', '', nan, nan]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")

In [ ]:
filepath = log_files[0]
print(filepath)

/home/intelligence/hustle/song_play_analysis/data/log_data/2018/11/2018-11-17-events.json


In [ ]:

df = pd.read_json(filepath, lines=True)
df.head(10)
# print(df.head())

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.40200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Rio De Janeiro Blue (Album Version),200,1542413200796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Breathe Underwater,200,1542413451796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
3,Poison The Well,Logged In,Chloe,F,56,Cuevas,184.60689,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Riverside,200,1542413675796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
4,Justin Bieber,Logged In,Chloe,F,57,Cuevas,196.88444,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,U Smile,200,1542413859796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
5,Type O Negative,Logged In,Chloe,F,58,Cuevas,366.23628,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Can't Lose You (Album Version),200,1542414055796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
6,The All-American Rejects,Logged In,Chloe,F,59,Cuevas,260.15302,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Happy Endings,200,1542414421796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
7,Eminem,Logged In,Mohammad,M,0,Rodriguez,252.73424,free,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,659,Still Don't Give A Fuck,200,1542414487796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
8,Silversun Pickups,Logged In,Chloe,F,60,Cuevas,353.61914,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Growing Old Is Getting Old (Album Version),200,1542414681796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
9,Kid Rock,Logged In,Chloe,F,61,Cuevas,396.72118,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,New Orleans (Album Version),200,1542415034796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df.page.eq("NextSong")]
df.head()
print(df.shape)

(127, 18)


In [ ]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-17 00:02:24.796
1   2018-11-17 00:06:40.796
2   2018-11-17 00:10:51.796
3   2018-11-17 00:14:35.796
4   2018-11-17 00:17:39.796
Name: ts, dtype: datetime64[ns]

In [ ]:
df["timestamp"] = t
df["hour"] = t.dt.hour
df["day"] = t.dt.day
df["week"] = t.dt.hour
df["month"] = t.dt.month
df["year"] = t.dt.year
df["day_name"] = t.dt.day_name()

In [ ]:
time_df = df[["timestamp", "hour", "day", "week", "month", "year", "day_name"]]
time_df.head()

,timestamp,hour,day,week,month,year,day_name
0,2018-11-17 00:02:24.796,0,17,0,11,2018,Saturday
1,2018-11-17 00:06:40.796,0,17,0,11,2018,Saturday
2,2018-11-17 00:10:51.796,0,17,0,11,2018,Saturday
3,2018-11-17 00:14:35.796,0,17,0,11,2018,Saturday
4,2018-11-17 00:17:39.796,0,17,0,11,2018,Saturday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df = user_df.drop_duplicates()

# Filter for empty user id
user_df = user_df[user_df['userId'] != '']

#Change column header names
user_df.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']

user_df.head()

,user_id,first_name,last_name,gender,level
0,49,Chloe,Cuevas,F,paid
7,88,Mohammad,Rodriguez,M,free
23,10,Sylvie,Cruz,F,free
24,54,Kaleb,Cook,M,free
32,33,Bronson,Harris,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
        print(results)
        print("YES")
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ( pd.to_datetime(row.ts), row.userId, row.level, 
    songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    


Kenny G


AttributeError: 'Series' object has no attribute 'user_id'

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.